In [1]:
import pyptvgtfs
import pandas as pd
import os

In [2]:
VERSIONS = ['20220403_025040',
'20220715_191941',
'20230805_030129',
'20231202_053358',
'20240229_224711',]

In [3]:
DFS = [pyptvgtfs.process_gtfs_zip(f'../downloads/{f}/gtfs.zip', f) for f in VERSIONS]
# Per file: 40s - 1m - 3m. 5 files: 2m - 5m

In [4]:
DFK = pd.concat(DFS, axis=0).set_index(['version_id', 'branch_id', 'table_name'])['df'].to_dict()

In [13]:
VERSIONS_2023 = ['20231021_105623',
'20231021_105623',
'20231030_224547',
'20231107_012112',
'20231113_155410',
'20231122_000348',
'20231130_234004',]
DFS_2023 = [pyptvgtfs.process_gtfs_zip(f'../downloads/{f}/gtfs.zip', f) for f in VERSIONS_2023]
# Per file: 40s - 1m - 3m. 5 files: 2m - 5m. 7 files: 3m - 5sm

In [22]:
VERSIONS_2023_24 = ['20231225_101846', '20240130_030040']
DFS_2023_24 = [pyptvgtfs.process_gtfs_zip(f'../downloads/{f}/gtfs.zip', f) for f in VERSIONS_2023_24]
# Per file: 40s - 1m - 3m. 5 files: 2m - 5m. 7 files: 3m - 5sm. 2 files: 1m 30s - 2m

In [23]:
DFK : dict[tuple, pd.DataFrame] = pd.concat(DFS_2023 + DFS + DFS_2023_24, axis=0).set_index(['version_id', 'branch_id', 'table_name'])['df'].to_dict()

In [54]:
DFK['20230805_030129', '4', 'trips'].head()

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,2-ain-mjp-2,T2,1.T2.2-ain-mjp-2.1.H,2-ain-mjp-2.1.H,Showgrounds/Flemington,0
1,2-ain-mjp-2,T3,1.T3.2-ain-mjp-2.4.R,2-ain-mjp-2.4.R,Flinders Street,1
2,2-ain-mjp-2,T2,10.T2.2-ain-mjp-2.2.H,2-ain-mjp-2.2.H,Showgrounds/Flemington,0
3,2-ain-mjp-2,T2,11.T2.2-ain-mjp-2.2.H,2-ain-mjp-2.2.H,Showgrounds/Flemington,0
4,2-ain-mjp-2,T2,12.T2.2-ain-mjp-2.2.H,2-ain-mjp-2.2.H,Showgrounds/Flemington,0


In [55]:
DFK['20231021_105623', '4', 'trips'].head()

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,2-ain-mjp-2,T2,1.T2.2-ain-mjp-2.3.R,2-ain-mjp-2.3.R,Flinders Street,1
1,2-ain-mjp-2,T3,1.T3.2-ain-mjp-2.2.R,2-ain-mjp-2.2.R,Flinders Street,1
2,2-ain-mjp-4,T2_1,1.T2.2-ain-mjp-4.2.R,2-ain-mjp-4.2.R,Flinders Street,1
3,2-ain-mjp-4,T3_1,1.T3.2-ain-mjp-4.1.R,2-ain-mjp-4.1.R,Flinders Street,1
4,2-ain-mjp-5,T2_3,1.T2.2-ain-mjp-5.2.R,2-ain-mjp-5.2.R,Flinders Street,1


In [ ]:
# It seems that after at least August 2023 and since at least October 2023, the service_id for buses ('4') has changed from normal service names like all others (T2_1) to specific service names to each route (MF1-12-831-aus). For trip id, before, it's 2-831--2-T2-1, now, it's 12-831--1-MF1-102. Seems like the format of trip_ids in 4 (use '-' instead of '.') being different from other operation branches (1,2...) is still consistent among versions.